# Import Libraries

In [2]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier

# Load Dataset

In [ ]:
data_path = r"E:\data_analyst_internship\E-commerce_return_rate_reductions\data"
file_name = "Ecommerce_Product_Sales.csv"  

df = pd.read_csv(os.path.join(data_path, file_name))
print("Dataset shape:", df.shape)
print(df.head())
print(df.columns)

Dataset shape: (500, 9)
  Product_ID     Product_Name        Category   Price  Units_Sold    Revenue  \
0  Product_1    Max Product_1            Toys  358.65         983  352552.95   
1  Product_2    Pro Product_2  Home & Kitchen  248.00          79   19592.00   
2  Product_3    Eco Product_3      Automotive  148.12         206   30512.72   
3  Product_4  Super Product_4     Electronics   15.76         585    9219.60   
4  Product_5  Ultra Product_5  Home & Kitchen   20.84         401    8356.84   

   Rating  Return_Rate Seller_Name  
0    3.63         0.16    Seller_B  
1    1.23         0.20    Seller_E  
2    2.09         0.05    Seller_D  
3    3.69         0.05    Seller_D  
4    2.01         0.15    Seller_B  
Index(['Product_ID', 'Product_Name', 'Category', 'Price', 'Units_Sold',
       'Revenue', 'Rating', 'Return_Rate', 'Seller_Name'],
      dtype='object')


# Data Cleaning

In [5]:
# Create binary target variable
df['is_returned'] = np.where(df['Return_Rate'] > 0, 1, 0)

# Drop original Return_Rate column
df.drop(columns=['Return_Rate'], inplace=True)

# Fill missing values
num_cols = df.select_dtypes(include=['int64','float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

cat_cols = df.select_dtypes(include=['object']).columns
for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

print("Cleaned dataset shape:", df.shape)

Cleaned dataset shape: (500, 9)


# Feature Engineering

In [6]:
# Total price feature
df['total_price'] = df['Price'] * df['Units_Sold']

# Select features
features = ['Category','Price','Units_Sold','Rating','Seller_Name','total_price']
X = df[features]
y = df['is_returned']

print("Selected features:", features)

Selected features: ['Category', 'Price', 'Units_Sold', 'Rating', 'Seller_Name', 'total_price']


# Train-Test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

Train shape: (400, 6) Test shape: (100, 6)


# XGBoost Pipeline

In [8]:
# Identify numeric and categorical columns
num_cols = X_train.select_dtypes(include=['int64','float64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns

# Preprocessing for categorical columns
preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# XGBoost classifier
xgb_model = Pipeline([
    ('prep', preprocessor),
    ('xgb', XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    ))
])

# Train the model
xgb_model.fit(X_train, y_train)

C:\Users\Muktha\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:158: UserWarning: [22:21:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['Price', 'Units_Sold', 'Rating', 'total_price'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Category', 'Seller_Name'], dtype='object'))])),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

# Model Evaluation

In [ ]:
y_pred = xgb_model.predict(X_test)
y_prob = xgb_model.predict_proba(X_test)[:,1]
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.98      1.00      0.99        98

    accuracy                           0.98       100
   macro avg       0.49      0.50      0.49       100
weighted avg       0.96      0.98      0.97       100

ROC-AUC: 0.7806122448979592


C:\Users\Muktha\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Muktha\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Muktha\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [ ]:
# Save High-Risk Products

# Add predicted probabilities
df['return_prob'] = xgb_model.predict_proba(X)[:,1]

# Output folder
output_path = r"E:\data_analyst_internship\E-commerce_return_rate_reductions\output"
os.makedirs(output_path, exist_ok=True)

# Save full dataset with probabilities
df.to_csv(os.path.join(output_path, "order_level_with_probs_xgb.csv"), index=False)

# Save high-risk products (prob > 0.7)
high_risk = df[df['return_prob'] > 0.7]
high_risk.to_csv(os.path.join(output_path, "product_return_high_risk_xgb.csv"), index=False)

print("✅ Files saved in:", output_path)

✅ Files saved in: E:\data_analyst_internship\E-commerce_return_rate_reductions\output


In [ ]:
#SQL

import pandas as pd
import sqlite3

# Load dataset
df = pd.read_csv(r"E:\data_analyst_internship\E-commerce_return_rate_reductions\data\Ecommerce_Product_Sales.csv")

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Write dataframe to SQL table
df.to_sql('orders', conn, index=False, if_exists='replace')

# Example 1: Return rate by category
query1 = """
SELECT Category,
       ROUND(AVG(Return_Rate)*100, 2) AS Return_Percentage
FROM orders
GROUP BY Category
ORDER BY Return_Percentage DESC;
"""
sql_result1 = pd.read_sql_query(query1, conn)
print("Return Rate by Category:")
print(sql_result1)

# Example 2: Average price of returned vs non-returned products
query2 = """
SELECT CASE WHEN Return_Rate > 0.5 THEN 'High Return' ELSE 'Low Return' END AS Return_Group,
       ROUND(AVG(Price), 2) AS Avg_Price
FROM orders
GROUP BY Return_Group;
"""
sql_result2 = pd.read_sql_query(query2, conn)
print("\nAverage Price: High vs Low Return Rate Products")
print(sql_result2)

# Example 3: Top 5 products with highest return probability
query3 = """
SELECT Product_Name, Category, ROUND(Return_Rate*100, 2) AS Return_Percentage
FROM orders
ORDER BY Return_Rate DESC
LIMIT 5;
"""
sql_result3 = pd.read_sql_query(query3, conn)
print("\nTop 5 Products with Highest Return Rate:")
print(sql_result3)

Return Rate by Category:
         Category  Return_Percentage
0          Sports              17.60
1          Beauty              17.53
2      Automotive              15.74
3     Electronics              15.49
4           Books              15.28
5        Clothing              14.80
6  Home & Kitchen              14.21
7            Toys              14.03

Average Price: High vs Low Return Rate Products
  Return_Group  Avg_Price
0   Low Return      252.1

Top 5 Products with Highest Return Rate:
        Product_Name     Category  Return_Percentage
0     Pro Product_88         Toys               30.0
1     Max Product_92  Electronics               30.0
2    Pro Product_118       Sports               30.0
3  Ultra Product_135       Sports               30.0
4    Pro Product_316   Automotive               30.0
